# Агрегатные функции 
## Юнит 5. РАБОТА С БАЗАМИ ДАННЫХ. SQL
### Skillfactory: DSPR-19

### 2.1. Основные агрегатные функции
 
Как и большинство других серверов реляционных баз данных, Postgres, которую мы используем в этом тренажере, поддерживает агрегатные функции. Она позволяет вычислить единственное значение, обработав множество строк.

Например, есть агрегатные функции, вычисляющие:

- count (количество не пустых значений),
- sum (сумму),
- avg (среднее),
- max (максимум) и
- min (минимум) для набора строк.


In [ ]:
select max(book_average_rating) as max_rating 
from books;

Запрос выдал нам лишь одну строку — максимальное значение колонки book_average_rating из таблицы books. Агрегатные функции используются, когда нужно посчитать параметры, общие для всех строк таблицы. 

In [ ]:
select max(book_average_rating) as max_rating
     , min(book_average_rating) as min_rating
     , avg(book_average_rating) as average_rating
     , sum(book_ratings_count) as books_ratings
from books;

Предположим, нам нужно вычислить максимальный, минимальный и средний рейтинг книг в таблице, а также сумму количества оценок всех книг и количество самих книг, но только для книг, language_code которых равен 'eng':



In [ ]:
select max(book_average_rating) as max_rating
     , min(book_average_rating) as min_rating
     , avg(book_average_rating) as average_rating
     , sum(book_ratings_count) as books_ratings
     , count(book_id) as books_count
from books
where language_code = 'eng';

Агрегатным функциям можно также передавать не просто столбцы таблиц, но и их арифметические комбинации:

In [ ]:
select avg(book_average_rating*book_average_rating + 5) as strange_rating
from books
where language_code = 'eng';

**Важно: агрегатные функции min, max, count можно использовать и для строковых типов данных, и для даты-времени.** 

С агрегатами можно работать так же, как и с обычными столбцами. Например, их можно перемножать или делить. Предположим, мы хотим получить среднее арифметическое по столбцу book_average_rating. Для этого мы можем просто использовать агрегатную функцию avg, а можем поделить сумму рейтингов на их количество:

Для подсчёта количества непустых строк можно использовать count(*):

In [ ]:
select count(*) not_null_strings_count
from books

### Задание 2.1.1
Напишите запрос, который находит максимальный book_id и размещает его в столбце с именем max_id (... as max_id) среди книг, у которых рейтинг меньше 4.

In [ ]:
select max(book_id) as max_id 
from books
where book_average_rating < 4

### Задание 2.1.2
Определите средний рейтинг (as average_rating) и минимальное количество оценок книги (min_ratings) для книг Стивена Кинга (Stephen King), изданных позднее 1990 года. Укажите запрос ниже:

In [ ]:
select avg(book_average_rating) as average_rating, min(book_ratings_count) as min_rating
from books
where author = 'Stephen King' and publishing_year > 1990

### Задание 2.1.3
Подсчитайте количество непустых строк в столбце book_name (не изменяйте название столбца) среди всех книг с рейтингом < 4 и количеством оценок > 100000.

In [ ]:
select count(book_name)
from books
where book_average_rating <4 and book_ratings_count > 100000

### Задание 2.1.4
Выведите минимальное значение столбца book_name для книг, у которых (language_code = 'eng' или language_code = 'en-US' и больше 10000 оценок), либо у которых (жанр 'fiction' или год публикации 1957). Скобки в условии указывают порядок действий.

In [ ]:
select min(book_name)
from books
where ((language_code = 'eng' or language_code = 'en-US') and book_ratings_count > 10000) or (genre = 'fiction' or publishing_year = 1957 )

### 2.2. DISTINCT и GROUP BY

Иногда нужны не все данные из таблицы, а только уникальные значения в столбце. Например, мы хотим получить всех авторов книг в таблице books. Для этого может использоваться команда DISTINCT:

In [ ]:
select distinct author
from books
order by author;

DISTINCT можно применять не только к одному столбцу, но и к нескольким сразу, тогда будут получены все уникальные комбинации столбцов в этих строках:

In [ ]:
select distinct author, publishing_year
from books
order by author, publishing_year;

DISTINCT можно применять не только для вывода уникальных значений, но и внутри агрегатных функций для того, чтобы они применялись только к уникальным элементам определённых столбцов. Например, если мы хотим посчитать количество авторов в нашей базе, то можно написать: 

In [ ]:
select count(distinct author) authors_count
from books;

### Задание 2.2.1
Сколько уникальных наименований книг содержится в базе?

Ответ: 1045

In [ ]:
select count(distinct book_name) book_name_count
from books;

GROUP BY используется для определения групп выходных строк, к которым могут применяться агрегатные функции (COUNT, MIN, MAX, AVG и SUM).

Если GROUP BY отсутствует, и используются агрегатные функции, то все столбцы с именами, упомянутыми в SELECT, должны быть включены в агрегатные функции. Эти функции будут применяться ко всему набору строк, которые удовлетворяют условию запроса. В противном случае все столбцы списка SELECT, не вошедшие в агрегатные функции, должны быть указаны в предложении GROUP BY.

В результате все выходные строки запроса разбиваются на группы, характеризуемые одинаковыми комбинациями значений в этих столбцах. После чего к каждой группе будут применены агрегатные функции. Следует иметь в виду, что для GROUP BY все значения NULL трактуются как равные, то есть при группировке по полю, содержащему NULL-значения, все такие строки попадут в одну группу. 

Давайте рассмотрим это на простом примере:

In [ ]:
select publishing_year, count(book_id)
from books
where publishing_year >= 2010
group by publishing_year 
order by publishing_year desc;

Мы сначала выделили только те книги, у которых год публикации больше или равен 2010, затем сгруппировали оставшиеся строки по publishing_year (отдельная группа для каждого publishing_year), потом посчитали количество книг (book_id) в каждой группе, а затем отсортировали по publishing_year по убыванию. 

GROUP BY можно использовать и без агрегатных функций. Тогда его действие будет равносильно действию DISTINCT.

Сравните вывод двух запросов в Metabase:

In [ ]:
select publishing_year
from books
group by publishing_year
order by publishing_year

In [ ]:
select distinct publishing_year
from books
order by publishing_year

GROUP BY можно использовать для любого количества столбцов (комбинаций столбцов) таблицы. Например, если мы хотим получить средние оценки и количество книг, вышедших с 2005 до 2010 года, в разбивке по жанру и году публикации книги, и отсортировать по убыванию по средней оценки книги в группе, мы можем написать так:



In [ ]:
select genre, publishing_year
     , avg(book_average_rating) books_rating
     , count(book_id) books_count
from books
where publishing_year between 2005 and 2010
group by genre, publishing_year
order by books_rating desc;

Важно: при использовании GROUP BY сортировка по столбцам вне выдачи невозможна. То есть в случае с последним запросом мы могли бы отсортировать данные только по genre, publishing_year, books_rating, books_count, а по колонке author уже не смогли бы. 

Для указания столбцов, по которым нужно производить группировку, можно использовать номера столбцов из select (по аналогии с order by) и по алиасам столбцов:

In [ ]:
select author, publishing_year > 1950 after_fifties, count(book_id)
from books
group by 1, after_fifties;

Итак, агрегатные функции, включенные в предложение SELECT запроса, не содержащего GROUP BY, исполняются над всеми результирующими строками этого запроса. Если же запрос содержит GROUP BY, то каждый набор строк, который имеет одинаковые значения столбца или группы столбцов, заданных в предложении GROUP BY, составляют группу, и агрегатные функции выполняются для каждой группы отдельно.

### Задание 2.2.2
1. Какой первый publishing_year выдаст запрос выше?

Ответ: -560

2. Узнайте среднее количество оценок (book_ratings_count), которые получили книги каждого отдельного автора в нашей базе. Вас интересуют данные о книгах, изданных в 2015 году или позже. Упорядочите данные по имени автора. Сколько в среднем оценок получили книги Харпер Ли? Ответ вводите без разделителей и пробелов.

Ответ: 138669


### Задание 2.2.3
Напишите запрос, который выводит топ-5 авторов по количеству написанных книг в период с 1985 до 2015 года.

Должны получиться столбцы author и books_count, отсортированные по books_count по убыванию и по author по возрастанию.

In [ ]:
select distinct author, 
    count(book_name) books_count
from books
where publishing_year between 1985 and 2015
group by author
order by books_count desc, author asc
limit 5

### Задание 2.2.4
Напишите запрос, который выводит топ 10 авторов, пишущих на английском языке (language_code = 'eng'), в максимальном количестве разных жанров?

Вывод должен содержать столбцы author, genres_count и быть отсортирован по genres_count по убыванию и по author по возрастанию.

In [ ]:
select author, 
    count(distinct genre) genres_count
from books
where language_code = 'eng' 
group by author
order by genres_count desc, author 
limit 10

### Задание 2.2.5
Напишите запрос, который выводит все уникальные жанры книг в базе данных на экран. (Итоговый столбец называется genre и не отсортирован, используйте DISTINCT, а не группировку.)

In [ ]:
select distinct genre
from books

### Задание 2.2.6
Напишите запрос, который выведет автора и количество книг, написанных автором в период с 1900 по 2000 годы. Отсортируйте сначала по количеству книг по убыванию, а затем по автору в алфавитом порядке. В получившемся списке выберите шестого.

Ответ: Author: Janet Evanovich, Book Count: 5 

### 2.3. HAVING
Если ключевое слово WHERE определяет фильтрацию строк, то ключевое слово HAVING применяется после группировки (GROUP BY) для определения аналогичной фильтрации, но по значениям агрегатных функций в группах.

Это необходимо для проверки значений, которые получены с помощью агрегатной функции не из отдельных строк источника записей, определенного в предложении FROM, а из групп таких строк. Поэтому такая проверка не может содержаться в предложении WHERE.

Например, если мы хотим получить список самых плодовитых авторов, которые написали больше 10 книг, мы можем написать:

In [ ]:
select author 
from books 
group by author 
having count(book_id) > 10;

В самом SELECT не обязательно указывать агрегатную функцию, которая используется в HAVING. Также заметим, что в предложении HAVING нельзя использовать псевдоним (например, books_count), используемый для именования значений агрегатной функции в предложении SELECT, так как обработка названий столбцов для вывода на экран производится позже, чем фильтрация значений агрегатных функций. 

Условия в HAVING можно комбинировать также, как и в where, то есть с использованием скобок, and и or: 

In [ ]:
select author, count(book_id) books_count
from books 
group by author 
having count(book_id) > 10 and avg(book_average_rating) > 3.8;

### Задание 2.3.1
Напишите запрос, который выводит издательства, опубликовавшие больше 100 книг.

Необходимые столбцы: publisher, сортировка в алфавитном порядке.

In [ ]:
select publisher
from books
group by publisher
having count(book_id) > 100
order by publisher asc

### Задание 2.3.2
Напишите запрос, который выводит список авторов, публиковавших книги с 1940 до 1980 годов в жанре fiction, средний рейтинг книг которых больше, чем 3.9.

Необходимые столбцы: author, average_rating. Сортировка по author по возрастанию.

In [ ]:
#рабочий запрос, не знаю, где ошибка
select author, avg(book_average_rating) average_rating 
from books
group by author, publishing_year, genre
having (publishing_year between 1940 and 1980) and genre = 'fiction' and avg(book_average_rating) > 3.9
order by author asc

### Задание 2.3.3
Напишите запрос, который выводит года публикации книг, жанр книг и их количество для каждой комбинации года публикации и жанра из таблицы, если средний рейтинг книг в этой группе меньше 4.2, а количество опубликованных книг больше 5.

Необходимые столбцы: publishing_year, genre, books_count с сортировкой по всем столбцам(в таком же порядке, как они перечислены в запросе) в порядке возрастания.

In [ ]:
select publishing_year, genre, count(book_id) as books_count
from books
group by publishing_year, genre
having avg(book_average_rating) < 4.2 and count(book_id) > 5
order by publishing_year, genre, books_count

### 2.4. Подводим итоги

### Задание 2.4.1
Определите среднее количество оценок, поставленных книгам авторов Stephen King или Amy Tan для книг, опубликованных с 1950 года до 1995 года (обязательно используйте BETWEEN!).

In [ ]:
select avg(book_ratings_count)
from books
where (author = 'Stephen King' or author = 'Amy Tan') and publishing_year between 1950 and 1995
group by author

### Задание 2.4.2
Напишите запрос, который получает средний рейтинг книг автора по различным жанрам. Выведите топ таких авторов с 3-го до 8-го места включительно. Должны получиться столбцы author, genre, average_rating, отсортированные по убыванию по среднему рейтингу и по возрастанию по автору и жанру.

In [ ]:
select author, genre, avg(book_average_rating) as average_rating
from books
group by author, genre
order by average_rating desc, author, genre
offset 2
limit 6


### Задание 2.4.3
Напишите запрос, который для каждого автора книг, опубликованных после 1930 года, подсчитывает количество издательств, в которых публиковались книги, сумму количества всех оценок таких книг, их средний рейтинг, минимальный и максимальный год публикации.

Выведите только таких авторов, которые публиковались хотя бы в двух издательствах и опубликовали хотя бы одну книгу после 1950 года.

In [ ]:
select author, count(distinct publisher), sum(book_ratings_count), avg(book_average_rating), min(publishing_year), max(publishing_year)
from books
where publishing_year > 1930
group by author
having count(distinct publisher) >=2 and max(publishing_year) > 1950